In [6]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 17 21:10:21 2020

@author: youxiandixiniu
"""

'''
SaleID	交易ID，唯一编码
name	汽车交易名称，已脱敏
regDate	汽车注册日期，例如20160101，2016年01月01日
model	车型编码，已脱敏
brand	汽车品牌，已脱敏
bodyType	车身类型：豪华轿车：0，微型车：1，厢型车：2，大巴车：3，敞篷车：4，双门汽车：5，商务车：6，搅拌车：7
fuelType	燃油类型：汽油：0，柴油：1，液化石油气：2，天然气：3，混合动力：4，其他：5，电动：6
gearbox	变速箱：手动：0，自动：1
power	发动机功率：范围 [ 0, 600 ]
kilometer	汽车已行驶公里，单位万km
notRepairedDamage	汽车有尚未修复的损坏：是：0，否：1
regionCode	地区编码，已脱敏
seller	销售方：个体：0，非个体：1
offerType	报价类型：提供：0，请求：1
creatDate	汽车上线时间，即开始售卖时间
price	二手车交易价格（预测目标）
v系列特征	匿名特征，包含v0-14在内15个匿名特征
'''
# In[]:
import pandas as pd
import datetime
import sys
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import xgboost as xgb
import re
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
#import pandas_profiling
color = sns.color_palette()
sns.set_style('darkgrid')
from math import isnan

import os
import sys
# 数据在Anaconda路径中；Jupyter代码在IDEA路径中；导入的自定义库是Anaconda路径中的库。 
# os.chdir设置之后，os.path也会变为该路径
os.chdir(r"E:\soft\Anaconda\Anaconda_Python3.6_code\data_analysis\100_Data_analysis_competition\3_TianChi\1_Used_car_transaction_price_prediction\data")

# print(sys.path)
BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))))
print(BASE_DIR)
sys.path.insert(0, os.path.join(BASE_DIR, "101_Sklearn\\7_SVM"))
print(sys.path)

import FeatureTools as ft
import Tools_customize as tc
import Binning_tools as bt

E:\soft\Anaconda\Anaconda_Python3.6_code\data_analysis
['E:\\soft\\Anaconda\\Anaconda_Python3.6_code\\data_analysis\\101_Sklearn\\7_SVM', 'e:\\soft\\anaconda\\anaconda_python3.6\\python36.zip', 'e:\\soft\\anaconda\\anaconda_python3.6\\DLLs', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib', 'e:\\soft\\anaconda\\anaconda_python3.6', '', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages\\pip-19.1-py3.6.egg', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages\\win32', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages\\win32\\lib', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages\\Pythonwin', 'e:\\soft\\anaconda\\anaconda_python3.6\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\dell\\.ipython']


In [7]:
# 一、表读取
train_data = ft.readFile_inputData('used_car_train_20200313.csv', parse_dates = ['regDate','creatDate'], sep=' ')
test_data = ft.readFile_inputData('used_car_testA_20200313.csv', parse_dates = ['regDate','creatDate'], sep=' ')

In [8]:
train_data

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
5,5,137642,20090602,24.0,10,0.0,1.0,0.0,109,10.0,...,0.260246,0.000518,0.119838,0.090922,0.048769,1.885526,-2.721943,2.457660,-0.286973,0.206573
6,6,2402,19990411,13.0,4,0.0,0.0,1.0,150,15.0,...,0.267998,0.117675,0.142334,0.025446,0.028174,-4.902200,1.610616,-0.834605,-1.996117,-0.103180
7,7,165346,19990706,26.0,14,1.0,0.0,0.0,101,15.0,...,0.239506,0.000000,0.122943,0.039839,0.082413,3.693829,-0.245014,-2.192810,0.236728,0.195567
8,8,2974,20030205,19.0,1,2.0,1.0,1.0,179,15.0,...,0.263833,0.116583,0.144255,0.039851,0.024388,-4.925234,1.587796,0.075348,-1.551098,0.069433
9,9,82021,19980101,7.0,7,5.0,0.0,0.0,88,15.0,...,0.262473,0.068267,0.012176,0.010291,0.098727,-1.089584,0.600683,-4.186210,0.198273,-1.025822
